Import libraries

In [459]:
import numpy as np
import pennylane as qml
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from pennylane import numpy as pnp

QDI layer

In [460]:
def qdi_layer(n_qubits, parameters, m, d):
    circuit = QuantumCircuit(n_qubits)
    param_index = 0

    
    for i in range(m):
        # Apply parameterized gates
        for qubit in range(n_qubits):
            circuit.rx(parameters[param_index], qubit)
            param_index += 1
        # Apply CNOT gates for entanglement
        for qubit in range(n_qubits - 1):
            circuit.cx(qubit, qubit + 1)
        circuit.cx(n_qubits - 1, 0)

    for i in range(d):
        for qubit in range(n_qubits):
            circuit.rz(parameters[param_index], qubit)
            param_index += 1

        for j in range(m):
            # Apply parameterized gates
            for qubit in range(n_qubits):
                circuit.rx(parameters[param_index], qubit)
                param_index += 1
            # Apply CNOT gates for entanglement
            for qubit in range(n_qubits - 1):
                circuit.cx(qubit, qubit + 1)
            circuit.cx(n_qubits - 1, 0)

    
    return circuit

PQN

In [461]:
def pqn_layer(n_qubits, parameters, i):
    circuit = QuantumCircuit(n_qubits)
    param_index = 0

    for qubit in range(n_qubits):
            circuit.rx(parameters[param_index], qubit)
            param_index += 1
    
    for k in range(i):
        # Apply parameterized gates
        for qubit in range(n_qubits):
            circuit.rz(parameters[param_index], qubit)
            param_index += 1
            circuit.ry(parameters[param_index], qubit)
            param_index += 1
            circuit.rz(parameters[param_index], qubit)
            param_index += 1
        # Apply CNOT gates for entanglement
        for qubit in range(n_qubits - 1):
            circuit.cx(qubit, qubit + 1)
        circuit.cx(n_qubits - 1, 0)
    
    return circuit


EFQ

Circuit function layer thing

In [462]:
def strong_layer(n_qubits, parameters, i):
    circuit = QuantumCircuit(n_qubits)
    param_index = 0
    
    for k in range(i):
        # Apply parameterized gates
        for qubit in range(n_qubits):
            circuit.rx(parameters[param_index], qubit)
            param_index += 1
            circuit.ry(parameters[param_index], qubit)
            param_index += 1
            circuit.rz(parameters[param_index], qubit)
            param_index += 1
        # Apply CNOT gates for entanglement
        for qubit in range(n_qubits - 1):
            circuit.cx(qubit, qubit + 1)
        circuit.cx(n_qubits - 1, 0)
    
    return circuit


Generate Bars and Stripes dataset

In [463]:
def get_bars_and_stripes(n):
    bitstrings = [list(np.binary_repr(i, n))[::-1] for i in range(2**n)]
    bitstrings = np.array(bitstrings, dtype=int)

    stripes = bitstrings.copy()
    stripes = np.repeat(stripes, n, 0)
    stripes = stripes.reshape(2**n, n * n)

    bars = bitstrings.copy()
    bars = bars.reshape(2**n * n, 1)
    bars = np.repeat(bars, n, 1)
    bars = bars.reshape(2**n, n * n)
    return np.vstack((stripes[0 : stripes.shape[0] - 1], bars[1 : bars.shape[0]]))


n = 3
size = n**2
n_qubits = size
data = get_bars_and_stripes(n)

bitstrings = []
nums = []
for d in data:
    bitstrings += ["".join(str(int(i)) for i in d)]
    nums += [int(bitstrings[-1], 2)]
probs = np.zeros(2**size)
probs[nums] = 1 / len(data)



Define the quantum circuit with Qiskit and Define the hybrid model in PennyLane

In [1]:
from pennylane import numpy as pnp
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from pennylane_qiskit import AerDevice


dev = AerDevice(wires=n_qubits, backend='aer_simulator')
#dev = qml.device("lightning.gpu", wires=n_qubits)

NameError: name 'n_qubits' is not defined

Circuit

In [ ]:
@qml.qnode(dev)
def circuit(parameters):
    #circuit = strong_layer(n_qubits, params, 6)
    #measurements = qml.from_qiskit(circuit)()
    param_index=0
    for i in range(4):
        for qubit in range(n_qubits):
            qml.RX(parameters[param_index], wires=qubit)
            param_index += 1
            qml.RY(parameters[param_index], wires=qubit)
            param_index += 1
            qml.RZ(parameters[param_index], wires=qubit)
            param_index += 1
        # Apply CNOT gates for entanglement
        for qubit in range(n_qubits - 1):
            qml.CNOT(wires=[qubit, qubit + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])
    
    return qml.probs(wires=range(n_qubits))

Cost function

In [ ]:
def cost(params):
    output_probs = circuit(params)
    
    target_probs = probs
    cost_value = np.sum( np.abs(target_probs-output_probs))
    return cost_value

MMD Cost function

In [ ]:

class MMD:

    def __init__(self, scales, space):
        gammas = 1 / (2 * (scales**2))
        sq_dists = pnp.abs(space[:, None] - space[None, :]) ** 2
        self.K = sum(pnp.exp(-gamma * sq_dists) for gamma in gammas) / len(scales)
        self.scales = scales

    def k_expval(self, px, py):
        # Kernel expectation value
        return px @ self.K @ py

    def __call__(self, px, py):
        pxy = px - py
        return self.k_expval(pxy, pxy)

bandwidth = np.array([0.25, 0.5, 1])
space = np.arange(2**n_qubits)

mmd = MMD(bandwidth, space)
def mmd_cost(params):
    output_probs = circuit(params)
    target_probs = probs
    loss = mmd.k_expval(target_probs, target_probs) - 2 * mmd.k_expval(target_probs, output_probs) + mmd.k_expval(output_probs, output_probs)
    return loss

Optimize the quantum circuit

In [ ]:
#init_params = np.random.random(n_qubits*4)
params = pnp.random.uniform(size=n_qubits*4*3, requires_grad=True)
#result = minimize(cost, init_params, method='COBYLA')
#optimal_params = result.x

# Evaluate and visualize the results
opt = qml.AdamOptimizer(stepsize=0.1)
steps = 100
history = []
for step in range(steps):
    params, prev_cost = opt.step_and_cost(mmd_cost, params)
    if step % 10 == 0:
        print(f"Step {step}, Cost: {mmd_cost(params)}")

    history.append(mmd_cost(params))
optimal_params = params
output_probs = circuit(optimal_params)

print(optimal_params)
print(output_probs)

Plot histograms

In [ ]:
probs = np.zeros(2**size)
probs[nums] = 1 / len(data)

plt.figure(figsize=(12, 5))
plt.bar(np.arange(2**size), probs, width=2.0, label=r"$\pi(x)$", color='blue')
plt.bar(np.arange(2**size), output_probs, width=2.0, label=r"$p(x)$",color='red')

plt.xticks(nums, bitstrings, rotation=80)

plt.xlabel("Samples")
plt.ylabel("Prob. Distribution")
plt.legend(loc="upper right")
plt.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
acr=0
for i in nums:
    acr+=output_probs[i]
print(acr)

In [ ]:
plt.plot(history)

plt.show()